# System Calls in Redis

An analysis of system calls in `redis`. This notebook aims to answer the following questions:

- Which system calls are made during a `redis-benchmark`?
- Which shared libraries do these system calls come from?
- If we aimed to implement compartmentalisation _along shared library boundaries_, what degree of privilege reduction would we see?

The tool used to take these measurements is also unproven. Any places where the `syso` tool has generated suspect data will be marked accordingly.

In [2]:
import pandas as pd
import json
import re

df = pd.read_json("./stats/redis-server-stats.json")
df.head()

,syscall_nr,library,pid,ppid,timestamp
0,13,unmapped,972706,972593,2131-07-29 14:41:55.479202
1,13,unmapped,972706,972593,2131-07-29 14:41:55.487866
2,13,unmapped,972706,972593,2131-07-29 14:41:55.489678
3,13,unmapped,972706,972593,2131-07-29 14:41:55.491246
4,13,unmapped,972706,972593,2131-07-29 14:41:55.492736


## Tool "Uptime"

The tool relies on a ring buffer to pass information from kernel space to userspace. If the ring buffer is full, then system calls will be missed.

Providing a number of syscall misses is possible but difficult to implement. The approach taken estimates "downtime", which was much simpler. If it turns out that a lot of syscalls are missed, some things can be done:
    - Increase ringbuffer memory (perhaps by a lot)
    - Optimise processing in the frontend to improve throughput (throw goroutines at the problem)

Below is a visualisation of uptime